In [93]:
import pandas as pd
import numpy as np


In [94]:
drivers_df = pd.read_csv('data/drivers.csv')
qualifying_df = pd.read_csv('data/qualifying.csv')
races_df = pd.read_csv('data/races.csv')
circuits_df = pd.read_csv('data/circuits.csv')


In [95]:
print(drivers_df.columns)
print(qualifying_df.columns)
print(races_df.columns)
print(circuits_df.columns)



Index(['driverId', 'driverRef', 'number', 'code', 'forename', 'surname', 'dob',
       'nationality', 'url'],
      dtype='object')
Index(['qualifyId', 'raceId', 'driverId', 'constructorId', 'number',
       'position', 'q1', 'q2', 'q3'],
      dtype='object')
Index(['raceId', 'year', 'round', 'circuitId', 'name', 'date', 'time', 'url',
       'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time',
       'quali_date', 'quali_time', 'sprint_date', 'sprint_time'],
      dtype='object')
Index(['circuitId', 'circuitRef', 'name', 'location', 'country', 'lat', 'lng',
       'alt', 'url'],
      dtype='object')


In [97]:
# Merge qualifying data with driver data
qualifying_df = qualifying_df.merge(drivers_df[['driverId', 'code', 'forename', 'surname']], 
                                     on='driverId', how='left', 
                                     suffixes=('', '_driver'))

qualifying_df.head()

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3,code,forename,surname,name,race_year,code_driver,forename_driver,surname_driver
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714,HAM,Lewis,Hamilton,Australian Grand Prix,2008.0,HAM,Lewis,Hamilton
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869,KUB,Robert,Kubica,Australian Grand Prix,2008.0,KUB,Robert,Kubica
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079,KOV,Heikki,Kovalainen,Australian Grand Prix,2008.0,KOV,Heikki,Kovalainen
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178,MAS,Felipe,Massa,Australian Grand Prix,2008.0,MAS,Felipe,Massa
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236,HEI,Nick,Heidfeld,Australian Grand Prix,2008.0,HEI,Nick,Heidfeld


In [98]:

# Merge qualifying data with race data (and avoid duplication of 'year' column)
qualifying_df = qualifying_df.merge(races_df[['raceId', 'name', 'year']], 
                                     on='raceId', how='left')

qualifying_df.head()

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3,code,forename,surname,name_x,race_year,code_driver,forename_driver,surname_driver,name_y,year
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714,HAM,Lewis,Hamilton,Australian Grand Prix,2008.0,HAM,Lewis,Hamilton,Australian Grand Prix,2008.0
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869,KUB,Robert,Kubica,Australian Grand Prix,2008.0,KUB,Robert,Kubica,Australian Grand Prix,2008.0
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079,KOV,Heikki,Kovalainen,Australian Grand Prix,2008.0,KOV,Heikki,Kovalainen,Australian Grand Prix,2008.0
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178,MAS,Felipe,Massa,Australian Grand Prix,2008.0,MAS,Felipe,Massa,Australian Grand Prix,2008.0
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236,HEI,Nick,Heidfeld,Australian Grand Prix,2008.0,HEI,Nick,Heidfeld,Australian Grand Prix,2008.0


In [99]:

# Ensure the 'year' column from the races_df is correctly selected
qualifying_df['race_year'] = qualifying_df['year']  # Rename 'year' to 'race_year' to avoid confusion

qualifying_df.head()

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3,code,forename,surname,name_x,race_year,code_driver,forename_driver,surname_driver,name_y,year
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714,HAM,Lewis,Hamilton,Australian Grand Prix,2008.0,HAM,Lewis,Hamilton,Australian Grand Prix,2008.0
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869,KUB,Robert,Kubica,Australian Grand Prix,2008.0,KUB,Robert,Kubica,Australian Grand Prix,2008.0
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079,KOV,Heikki,Kovalainen,Australian Grand Prix,2008.0,KOV,Heikki,Kovalainen,Australian Grand Prix,2008.0
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178,MAS,Felipe,Massa,Australian Grand Prix,2008.0,MAS,Felipe,Massa,Australian Grand Prix,2008.0
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236,HEI,Nick,Heidfeld,Australian Grand Prix,2008.0,HEI,Nick,Heidfeld,Australian Grand Prix,2008.0


In [100]:

# Drop the original 'year' column to avoid duplicate column issues
qualifying_df = qualifying_df.drop(columns=['year'])

# Filter to select years 2021, 2022, 2023, and 2024
qualifying_filtered = qualifying_df[qualifying_df['race_year'].isin([2021, 2022, 2023, 2024])]

qualifying_filtered.head()

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3,code,forename,surname,name_x,race_year,code_driver,forename_driver,surname_driver,name_y
8694,8735,1052,830,9,33,1,1:30.499,1:30.318,1:28.997,VER,Max,Verstappen,Bahrain Grand Prix,2021.0,VER,Max,Verstappen,Bahrain Grand Prix
8695,8736,1052,1,131,44,2,1:30.617,1:30.085,1:29.385,HAM,Lewis,Hamilton,Bahrain Grand Prix,2021.0,HAM,Lewis,Hamilton,Bahrain Grand Prix
8696,8737,1052,822,131,77,3,1:31.200,1:30.186,1:29.586,BOT,Valtteri,Bottas,Bahrain Grand Prix,2021.0,BOT,Valtteri,Bottas,Bahrain Grand Prix
8697,8738,1052,844,6,16,4,1:30.691,1:30.010,1:29.678,LEC,Charles,Leclerc,Bahrain Grand Prix,2021.0,LEC,Charles,Leclerc,Bahrain Grand Prix
8698,8739,1052,842,213,10,5,1:30.848,1:30.513,1:29.809,GAS,Pierre,Gasly,Bahrain Grand Prix,2021.0,GAS,Pierre,Gasly,Bahrain Grand Prix


In [101]:

# Drop columns with unnecessary suffixes and duplicates
qualifying_filtered = qualifying_filtered.drop(columns=['name_x', 'code_driver', 'forename_driver', 'surname_driver'])

# Rename columns to remove the '_y' suffix where needed
qualifying_filtered = qualifying_filtered.rename(columns={
    'name_y': 'race_name',
    'code': 'driver_code',
    'forename': 'driver_name',
    'surname': 'driver_surname'
})

# Show the updated columns
qualifying_filtered.head()


,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3,driver_code,driver_name,driver_surname,race_year,race_name
8694,8735,1052,830,9,33,1,1:30.499,1:30.318,1:28.997,VER,Max,Verstappen,2021.0,Bahrain Grand Prix
8695,8736,1052,1,131,44,2,1:30.617,1:30.085,1:29.385,HAM,Lewis,Hamilton,2021.0,Bahrain Grand Prix
8696,8737,1052,822,131,77,3,1:31.200,1:30.186,1:29.586,BOT,Valtteri,Bottas,2021.0,Bahrain Grand Prix
8697,8738,1052,844,6,16,4,1:30.691,1:30.010,1:29.678,LEC,Charles,Leclerc,2021.0,Bahrain Grand Prix
8698,8739,1052,842,213,10,5,1:30.848,1:30.513,1:29.809,GAS,Pierre,Gasly,2021.0,Bahrain Grand Prix


In [102]:

# Filter to select years 2021, 2022, 2023, and 2024
qualifying_filtered = qualifying_df[qualifying_df['race_year'].isin([2021, 2022, 2023, 2024])]

# Drop columns with unnecessary suffixes and duplicates
qualifying_filtered = qualifying_filtered.drop(columns=['name_x', 'code_driver', 'forename_driver', 'surname_driver'])

# Rename columns to remove the '_y' suffix where needed
qualifying_filtered = qualifying_filtered.rename(columns={
    'name_y': 'race_name',
    'code': 'driver_code',
    'forename': 'driver_name',
    'surname': 'driver_surname'
})

# Show the updated columns
qualifying_filtered.head()


,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3,driver_code,driver_name,driver_surname,race_year,race_name
8694,8735,1052,830,9,33,1,1:30.499,1:30.318,1:28.997,VER,Max,Verstappen,2021.0,Bahrain Grand Prix
8695,8736,1052,1,131,44,2,1:30.617,1:30.085,1:29.385,HAM,Lewis,Hamilton,2021.0,Bahrain Grand Prix
8696,8737,1052,822,131,77,3,1:31.200,1:30.186,1:29.586,BOT,Valtteri,Bottas,2021.0,Bahrain Grand Prix
8697,8738,1052,844,6,16,4,1:30.691,1:30.010,1:29.678,LEC,Charles,Leclerc,2021.0,Bahrain Grand Prix
8698,8739,1052,842,213,10,5,1:30.848,1:30.513,1:29.809,GAS,Pierre,Gasly,2021.0,Bahrain Grand Prix


In [103]:


# Create a dictionary for track features (Example)
track_features = {
    'French Grand Prix': {'altitude': 135, 'track_speed': 'medium', 'track_corners': 'fast', 'elevation_change': 'low'},
    'Chinese Grand Prix': {'altitude': 4, 'track_speed': 'medium', 'track_corners': 'tight', 'elevation_change': 'medium'},
    'Portuguese Grand Prix': {'altitude': 200, 'track_speed': 'medium', 'track_corners': 'fast', 'elevation_change': 'high'},
    'Bahrain Grand Prix': {'altitude': 3, 'track_speed': 'medium', 'track_corners': 'tight', 'elevation_change': 'low'},
    'Saudi Arabian Grand Prix': {'altitude': 5, 'track_speed': 'high', 'track_corners': 'fast', 'elevation_change': 'low'},
    'Australian Grand Prix': {'altitude': 26, 'track_speed': 'medium', 'track_corners': 'tight', 'elevation_change': 'low'},
    'Azerbaijan Grand Prix': {'altitude': 28, 'track_speed': 'high', 'track_corners': 'tight', 'elevation_change': 'low'},
    'Miami Grand Prix': {'altitude': 1, 'track_speed': 'medium', 'track_corners': 'fast', 'elevation_change': 'low'},
    'Emilia Romagna Grand Prix': {'altitude': 63, 'track_speed': 'medium', 'track_corners': 'tight', 'elevation_change': 'high'},
    'Monaco Grand Prix': {'altitude': 0, 'track_speed': 'low', 'track_corners': 'tight', 'elevation_change': 'high'},
    'Spanish Grand Prix': {'altitude': 120, 'track_speed': 'medium', 'track_corners': 'tight', 'elevation_change': 'medium'},
    'Austrian Grand Prix': {'altitude': 673, 'track_speed': 'high', 'track_corners': 'fast', 'elevation_change': 'high'},
    'British Grand Prix': {'altitude': 165, 'track_speed': 'high', 'track_corners': 'fast', 'elevation_change': 'low'},
    'Canadian Grand Prix': {'altitude': 15, 'track_speed': 'high', 'track_corners': 'tight', 'elevation_change': 'low'},
    'Hungarian Grand Prix': {'altitude': 150, 'track_speed': 'low', 'track_corners': 'tight', 'elevation_change': 'low'},
    'Belgian Grand Prix': {'altitude': 385, 'track_speed': 'high', 'track_corners': 'fast', 'elevation_change': 'high'},
    'Dutch Grand Prix': {'altitude': 4, 'track_speed': 'medium', 'track_corners': 'fast', 'elevation_change': 'high'},
    'Italian Grand Prix': {'altitude': 142, 'track_speed': 'high', 'track_corners': 'fast', 'elevation_change': 'low'},
    'Singapore Grand Prix': {'altitude': 0, 'track_speed': 'low', 'track_corners': 'tight', 'elevation_change': 'low'},
    'Japanese Grand Prix': {'altitude': 23, 'track_speed': 'high', 'track_corners': 'fast', 'elevation_change': 'high'},
    'Qatar Grand Prix': {'altitude': 10, 'track_speed': 'high', 'track_corners': 'fast', 'elevation_change': 'low'},
    'United States Grand Prix': {'altitude': 430, 'track_speed': 'medium', 'track_corners': 'tight', 'elevation_change': 'high'},
    'Mexico City Grand Prix': {'altitude': 2240, 'track_speed': 'high', 'track_corners': 'fast', 'elevation_change': 'low'},
    'São Paulo Grand Prix': {'altitude': 800, 'track_speed': 'medium', 'track_corners': 'fast', 'elevation_change': 'high'},
    'Las Vegas Grand Prix': {'altitude': 2, 'track_speed': 'high', 'track_corners': 'fast', 'elevation_change': 'low'},
    'Abu Dhabi Grand Prix': {'altitude': 3, 'track_speed': 'medium', 'track_corners': 'tight', 'elevation_change': 'low'},
    'Turkish Grand Prix': {'altitude': 100, 'track_speed': 'medium', 'track_corners': 'fast', 'elevation_change': 'high'}
}


# Function to add track features
def add_track_features(row):
    track_name = row['race_name']
    if track_name in track_features:
        features = track_features[track_name]
        row['track_speed'] = features['track_speed']
        row['track_corners'] = features['track_corners']
        row['elevation_change'] = features['elevation_change']
        row['altitude'] = features['altitude']
    return row

# Apply the function to add track features
qualifying_filtered = qualifying_filtered.apply(add_track_features, axis=1)

# Select final columns
qualifying_filtered = qualifying_filtered[['driver_code', 'driver_name', 'driver_surname', 'race_name', 'race_year', 
                                           'q1', 'q2', 'q3', 'track_speed', 'track_corners', 
                                           'elevation_change', 'altitude', 'position']]

print(qualifying_filtered.columns)


Index(['driver_code', 'driver_name', 'driver_surname', 'race_name',
       'race_year', 'q1', 'q2', 'q3', 'track_speed', 'track_corners',
       'elevation_change', 'altitude', 'position'],
      dtype='object')


In [104]:
# Rename columns to match the desired output
quali_data = qualifying_filtered.rename(columns={
    'driver_code': 'driver_code',
    'driver_name': 'driver_name',
    'driver_surname': 'driver_surname',
    'race_name': 'race_name',
    'race_year': 'race_year',
    'q1': 'q1_time',
    'q2': 'q2_time',
    'q3': 'q3_time',
    'position': 'final_position'
})

# Show the resulting dataset
quali_data.head()

,driver_code,driver_name,driver_surname,race_name,race_year,q1_time,q2_time,q3_time,track_speed,track_corners,elevation_change,altitude,final_position
8694,VER,Max,Verstappen,Bahrain Grand Prix,2021.0,1:30.499,1:30.318,1:28.997,medium,tight,low,3.0,1
8695,HAM,Lewis,Hamilton,Bahrain Grand Prix,2021.0,1:30.617,1:30.085,1:29.385,medium,tight,low,3.0,2
8696,BOT,Valtteri,Bottas,Bahrain Grand Prix,2021.0,1:31.200,1:30.186,1:29.586,medium,tight,low,3.0,3
8697,LEC,Charles,Leclerc,Bahrain Grand Prix,2021.0,1:30.691,1:30.010,1:29.678,medium,tight,low,3.0,4
8698,GAS,Pierre,Gasly,Bahrain Grand Prix,2021.0,1:30.848,1:30.513,1:29.809,medium,tight,low,3.0,5


In [105]:
quali_data.describe()

,race_year,altitude,final_position
count,1558.000000,1518.000000,1558.000000
mean,2022.307445,229.100132,10.487805
std,1.041136,462.146959,5.761770
min,2021.000000,0.000000,1.000000
25%,2021.000000,4.000000,5.250000
50%,2022.000000,28.000000,10.000000
75%,2023.000000,165.000000,15.000000
max,2024.000000,2240.000000,20.000000
